In [108]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [109]:
dataPath = 'data'
for file in os.listdir(dataPath):
    if file.endswith('.xlsx'):
        tempDF = pd.read_excel(dataPath+'/'+file)
        tempDF.to_csv(dataPath+'/'+os.path.splitext(file)[0]+'.csv', index = False)

In [121]:
mobiDF = pd.DataFrame()
for file in os.listdir(dataPath):
    if file.endswith('.csv'):
        tempDF = pd.read_csv(dataPath+'/'+file)
        mobiDF = mobiDF.append(tempDF)
mobiDF.reset_index(drop = True, inplace = True)

In [123]:
mobiDF

,Departure,Return,Account,Bike,Departure station,Return station,Membership type,Covered distance (m),Duration (sec.),Departure battery voltage (mV),...,Departure slot,Return slot,Departure temperature (deg C),Return temperature (deg C),Departure temperature (C),Return temperature (C),Stopover duration,Manager,Number of bike stopovers,Return temperature (Â°C)
0,2018-01-01 00:00:00,2018-01-01 00:00:00,ID00001,754.0,0048 Comox & Broughton,0002 Burrard Station,365 Day Founding Plus,1382.0,402,3715.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:00:00,2018-01-01 00:00:00,ID00001,754.0,0048 Comox & Broughton,0048 Comox & Broughton,365 Day Founding Plus,0.0,17,3797.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:00:00,2018-01-01 00:00:00,ID00002,517.0,0093 Chilco & Robson,0035 Coal Harbour Community Centre,365 Day Founding Standard,1960.0,1015,3645.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 00:00:00,2018-01-01 00:00:00,ID00003,231.0,0071 14th & Granville,0211 14th & Alder,365 Corporate Plus,624.0,194,3650.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 00:00:00,2018-01-01 00:00:00,ID00004,1302.0,0187 Bute & Davie,0079 Nelson & Richards,365 Day Founding Plus,1756.0,571,3686.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908475,2021-04-01 0:00,2021-04-01 1:00,NaN,1272.0,0278 15th & Victoria (Trout Lake),0228 Keefer & Hawks,365 Day Pass Plus SALE,3904.0,1197,3752.0,...,NaN,NaN,NaN,NaN,7.0,10.0,NaN,NaN,0.0,NaN
2908476,2021-04-01 0:00,2021-04-01 0:00,NaN,875.0,0126 Thurlow & Pendrell,0138 Richards & Helmcken,30 Day Pass,971.0,361,3997.0,...,NaN,NaN,NaN,NaN,11.0,13.0,NaN,NaN,0.0,NaN
2908477,2021-04-01 0:00,2021-04-01 0:00,NaN,380.0,0060 Bidwell & Burnaby,0045 Beach & Hornby,30 Day Pass,1502.0,463,3586.0,...,NaN,NaN,NaN,NaN,9.0,11.0,NaN,NaN,0.0,NaN
2908478,2021-04-01 0:00,2021-04-01 0:00,NaN,668.0,0060 Bidwell & Burnaby,0045 Beach & Hornby,30 Day Pass,1462.0,485,3495.0,...,NaN,NaN,NaN,NaN,9.0,11.0,NaN,NaN,0.0,NaN


In [124]:
def nullPercent(df):
    # Create a temp dataframe for null vlaue totals
    nullTotalDF = pd.DataFrame(df.isnull().sum()).reset_index()
    nullTotalDF.columns = ['Category', 'Total Null']
    # Create a temp dataframe for null value percentages
    nullPercentDF = pd.DataFrame(round(df.isnull().sum()/df.shape[0]*100,2)).reset_index()
    nullPercentDF.columns = ['Category', 'Percent Null (%)']
    # Merge temporary null value datasets and sort by null percentage
    nullDF = pd.merge(nullTotalDF, nullPercentDF, on = 'Category')
    display(nullDF.sort_values('Percent Null (%)', ascending = False))

In [125]:
nullPercent(mobiDF)

,Category,Total Null,Percent Null (%)
26,Return temperature (Â°C),2876218,98.89
20,Return temperature (deg C),2869232,98.65
19,Departure temperature (deg C),2869232,98.65
15,Formula,2862692,98.43
23,Stopover duration,2845627,97.84
24,Manager,2782010,95.65
17,Departure slot,2719422,93.50
18,Return slot,2719155,93.49
25,Number of bike stopovers,2619601,90.07
16,Membership Type,2559468,88.00


In [126]:
mobiDF = pd.DataFrame()
for file in os.listdir(dataPath):
    if file.endswith('.csv'):
        tempDF = pd.read_csv(dataPath+'/'+file)
        for c in tempDF.columns:
            if c == 'Return temperature (Â°C)':
                tempDF.rename(columns = {'Return temperature (Â°C)':'Return temperature (°C)'}, inplace = True)
            elif c == 'Formula':
                tempDF.rename(columns = {'Formula':'Membership type'}, inplace = True)
            elif c == 'Return temperature (deg C)':
                tempDF.rename(columns = {'Return temperature (deg C)':'Return temperature (°C)'}, inplace = True)
            elif c == 'Departure temperature (deg C)':
                tempDF.rename(columns = {'Departure temperature (deg C)':'Departure temperature (°C)'}, inplace = True)
            elif c == 'Membership Type':
                tempDF.rename(columns = {'Membership Type':'Membership type'}, inplace = True)
            elif c == 'Departure temperature (C)':
                tempDF.rename(columns = {'Departure temperature (C)':'Departure temperature (°C)'}, inplace = True)
            elif c == 'Return temperature (C)':
                tempDF.rename(columns = {'Return temperature (C)':'Return temperature (°C)'}, inplace = True)
            elif c == 'Number of bike stopovers':
                tempDF.rename(columns = {'Number of bike stopovers':'Number of stopovers'}, inplace = True)
            elif c == 'Stopover duration':
                tempDF.rename(columns = {'Stopover duration':'Stopover duration (sec.)'}, inplace = True)
        mobiDF = mobiDF.append(tempDF)
mobiDF.reset_index(drop = True, inplace = True)

In [127]:
nullPercent(mobiDF)

,Category,Total Null,Percent Null (%)
17,Manager,2782010,95.65
15,Departure slot,2719422,93.50
16,Return slot,2719155,93.49
2,Account,2335036,80.28
10,Return battery voltage (mV),53521,1.84
9,Departure battery voltage (mV),53521,1.84
3,Bike,1190,0.04
6,Membership type,498,0.02
1,Return,153,0.01
5,Return station,253,0.01


In [128]:
mobiDF.drop(columns = ['Manager', 'Departure slot', 'Return slot', 'Account', 'Departure battery voltage (mV)', 'Return battery voltage (mV)'], inplace = True)

In [129]:
nullPercent(mobiDF)

,Category,Total Null,Percent Null (%)
2,Bike,1190,0.04
5,Membership type,498,0.02
1,Return,153,0.01
4,Return station,253,0.01
0,Departure,0,0.00
3,Departure station,9,0.00
6,Covered distance (m),0,0.00
7,Duration (sec.),0,0.00
8,Departure temperature (°C),0,0.00
9,Return temperature (°C),0,0.00


In [130]:
mobiDF.dropna(inplace = True)

In [131]:
nullPercent(mobiDF)

,Category,Total Null,Percent Null (%)
0,Departure,0,0.0
1,Return,0,0.0
2,Bike,0,0.0
3,Departure station,0,0.0
4,Return station,0,0.0
5,Membership type,0,0.0
6,Covered distance (m),0,0.0
7,Duration (sec.),0,0.0
8,Departure temperature (°C),0,0.0
9,Return temperature (°C),0,0.0


In [132]:
mobiDF

,Departure,Return,Bike,Departure station,Return station,Membership type,Covered distance (m),Duration (sec.),Departure temperature (°C),Return temperature (°C),Stopover duration (sec.),Number of stopovers
0,2018-01-01 00:00:00,2018-01-01 00:00:00,754.0,0048 Comox & Broughton,0002 Burrard Station,365 Day Founding Plus,1382.0,402,2,4,0,0
1,2018-01-01 00:00:00,2018-01-01 00:00:00,754.0,0048 Comox & Broughton,0048 Comox & Broughton,365 Day Founding Plus,0.0,17,2,1,0,0
2,2018-01-01 00:00:00,2018-01-01 00:00:00,517.0,0093 Chilco & Robson,0035 Coal Harbour Community Centre,365 Day Founding Standard,1960.0,1015,0,1,0,0
3,2018-01-01 00:00:00,2018-01-01 00:00:00,231.0,0071 14th & Granville,0211 14th & Alder,365 Corporate Plus,624.0,194,0,1,0,0
4,2018-01-01 00:00:00,2018-01-01 00:00:00,1302.0,0187 Bute & Davie,0079 Nelson & Richards,365 Day Founding Plus,1756.0,571,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2908475,2021-04-01 0:00,2021-04-01 1:00,1272.0,0278 15th & Victoria (Trout Lake),0228 Keefer & Hawks,365 Day Pass Plus SALE,3904.0,1197,7,10,0,0
2908476,2021-04-01 0:00,2021-04-01 0:00,875.0,0126 Thurlow & Pendrell,0138 Richards & Helmcken,30 Day Pass,971.0,361,11,13,0,0
2908477,2021-04-01 0:00,2021-04-01 0:00,380.0,0060 Bidwell & Burnaby,0045 Beach & Hornby,30 Day Pass,1502.0,463,9,11,0,0
2908478,2021-04-01 0:00,2021-04-01 0:00,668.0,0060 Bidwell & Burnaby,0045 Beach & Hornby,30 Day Pass,1462.0,485,9,11,0,0
